In [1]:
!pip install -U git+https://github.com/faustomorales/keras-ocr.git
!pip install imgaug
!pip install -U opencv-python
%tensorflow_version 2.x

!gdown --id 12WOvX4IDbmJUTDOdWbV3wWAc4lr5GGmO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/faustomorales/keras-ocr.git to /tmp/pip-req-build-9v_d8v1g
  Running command git clone -q https://github.com/faustomorales/keras-ocr.git /tmp/pip-req-build-9v_d8v1g
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 930 kB 8.8 MB/s 
     |████████████████████████████████| 9.5 MB 50.5 MB/s 
     |████████████████████████████████| 603 kB 60.7 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.9.0.post2+40bb9bd-py3-none-any.whl size=42441 sha256=395ed38f34cb967c37e2cf7ce5f322a5231d7070ae68addf74cc2ca4a5851340
  Stored in directory: /tmp/pip-ephem-wheel-cache-r9pw2p4a/wheels/20/d0/06/ed290995144ec8dde71d3a91d0d590145445c01be60ea70c9b
  Created wheel for validators: filename=validators-0.19

In [2]:
import random
import string
import math
import itertools
import os
import zipfile
import typing

import numpy as np

import imgaug
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection

import keras_ocr

dir = "."

In [3]:
def extract_train_dataset(main_dir="."):
  if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
  train_image_dir = os.path.join(main_dir, "train_images")
  if not os.path.isdir(train_image_dir):
    os.mkdir(train_image_dir)
  training_zip_images_path = os.path.join(".", "Challenge2_Training_Task3_Images_GT.zip")
  if len(os.listdir(train_image_dir)) != 849:
    with zipfile.ZipFile(training_zip_images_path, 'r') as zip_ref:
      zip_ref.extractall(train_image_dir)

In [4]:
def preprocess_train_dataset(dir="."):
  main_dir = os.path.join(dir, "icdar2013")
  extract_train_dataset(main_dir)
  dataset = []

  train_image_dir = os.path.join(main_dir, "train_images")
  train_gt_path = os.path.join(train_image_dir, "gt.txt")

  dataset = []

  with open(train_gt_path, "r", encoding="utf8") as f:
    for line in f:
      image_id, text = line.split(".png,")
      text = text.strip("\"\n ").lower()
      image_path = os.path.join(train_image_dir, image_id + ".png")
      dataset.append((image_path, None, text))
      
  return dataset

In [5]:
dataset = preprocess_train_dataset(dir)
train_labels = [(filepath, box, word.lower()) for filepath, box, word in dataset]

In [6]:
recognizer = keras_ocr.recognition.Recognizer()
recognizer.compile()

Looking for /root/.keras-ocr/crnn_kurapan.h5


In [10]:
batch_size = 8
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.GammaContrast(gamma=(0.25, 3.0)),
])

train_labels, validation_labels = sklearn.model_selection.train_test_split(train_labels, train_size=0.8, random_state=42)

(training_image_gen, training_steps), (validation_image_gen, validation_steps) = [
    (
        keras_ocr.datasets.get_recognizer_image_generator(
            labels=labels,
            height=recognizer.model.input_shape[1],
            width=recognizer.model.input_shape[2],
            alphabet=recognizer.alphabet,
            augmenter=augmenter
        ),
        len(labels) // batch_size
    ) for labels, augmenter in [(train_labels, augmenter), (validation_labels, None)]     
]
training_gen, validation_gen = [
    recognizer.get_batch_generator(
        image_generator=image_generator,
        batch_size=batch_size
    )
    for image_generator in [training_image_gen, validation_image_gen]
]

In [11]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False),
    tf.keras.callbacks.ModelCheckpoint('recognizer_icdar2013.h5', monitor='val_loss', save_best_only=True),
]
recognizer.training_model.fit(
    x=training_gen,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    validation_data=validation_gen,
    callbacks=callbacks,
    epochs=50,
    verbose=1
)

55 / 542 instances have illegal characters.
Epoch 1/50
67/67 [==============================] - 11s 141ms/step - loss: 3.3091 - val_loss: 6.8293
Epoch 2/50
67/67 [==============================] - 7s 101ms/step - loss: 1.4146 - val_loss: 2.7407
Epoch 3/50
67/67 [==============================] - 7s 101ms/step - loss: 1.1770 - val_loss: 2.6569
Epoch 4/50
67/67 [==============================] - 6s 97ms/step - loss: 1.2884 - val_loss: 3.3323
Epoch 5/50
67/67 [==============================] - 7s 104ms/step - loss: 0.9084 - val_loss: 1.9994
Epoch 6/50
67/67 [==============================] - 7s 99ms/step - loss: 0.6966 - val_loss: 2.5109
Epoch 7/50
67/67 [==============================] - 7s 98ms/step - loss: 0.7671 - val_loss: 4.2591
Epoch 8/50
67/67 [==============================] - 7s 99ms/step - loss: 0.6334 - val_loss: 3.0481
Epoch 9/50
67/67 [==============================] - 7s 101ms/step - loss: 0.6178 - val_loss: 3.3749
Epoch 10/50
67/67 [==============================] - 7s 103

In [12]:
from google.colab import files
files.download('/content/recognizer_icdar2013.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>